In [21]:
import pandas as pd
import csv
from matplotlib import pyplot as plt
import os
import numpy as np
from scipy import stats


In [22]:
R=0.4

In [23]:
folder_list=list()
current_folder = os.getcwd()
for item in os.listdir(current_folder):
    if len(item) <= 3:
        folder_list.append(item)
folder_list.sort(key=lambda x:int(x.replace("G","")))

In [24]:
csv_list=list()
for folder in folder_list:
    folder_path=os.path.join(current_folder,folder)
    csv_list.append(os.listdir(folder_path))
gain_list=list()
for folder in folder_list:
    gain_list.append(int(folder.replace("G","")))

In [25]:
sum=0
for csv_file in csv_list:
    sum+=len(csv_file)
print(sum)

82


In [26]:
I_mm = list()
I_adc = list()
G = list()
G_LSB = list()

In [27]:
for i in range(len(csv_list)):
    for csv_file in csv_list[i]:

        G.append(gain_list[i])
        G_LSB.append(4096/2**15/gain_list[i])
        splitted_I = csv_file.replace(".csv", "").split(" ")
        if len(splitted_I) == 2:
            I_mm.append(float(splitted_I[0]) +
                float(splitted_I[1])/(10**len(splitted_I[1])))
        else:
            I_mm.append(float(splitted_I[0]))

        folder_path=os.path.join(current_folder,folder_list[i])
        with open(os.path.join(folder_path,csv_file), "r") as f:
            csv_reader = csv.reader(f, delimiter=',')
            I_adc_temp = list()
            for row in csv_reader:
                # cek kalo list kosong atau engga
                if row:
                    I_adc_temp.append(float(row[0]))
        I_adc.append(np.average(I_adc_temp))

In [28]:
Idata = pd.DataFrame([I_mm, I_adc, G, G_LSB]).T
Idata.columns = ['I multimeter', 'I ADC', 'G', 'G LSB']
Idata=Idata.sort_values(["G","I multimeter"]).reset_index(drop=True)
#Idata = Idata.sort_values(['G','I ADC'])
#Idata['I ADC'] = Idata['I ADC']*3/6.2/1000

In [29]:
Idata['I ADC'] = Idata['I ADC'].multiply(Idata['G LSB'], axis="index")

\begin{equation*}
  V_{adc}=I_{in}R_{shunt}\frac{1}{3}A_v
\end{equation*}
\begin{equation*}
    I_{in}=\frac{3V_{adc}}{R_{shunt}A_v}
\end{equation*}


In [30]:
Slope_I = list()
Inter_I = list()
for gain in gain_list:
    reg_result= stats.linregress(Idata['I ADC'].loc[Idata['G']==gain], Idata['I multimeter'].loc[Idata['G']==gain])
    Slope_I.append(reg_result.slope)
    Inter_I.append(reg_result.intercept)


In [31]:
Idata['I multimeter'].loc[Idata['G']==gain]

67     20.26
68     33.35
69     44.14
70     54.51
71     61.70
72     71.30
73     81.00
74     91.40
75    101.30
76    112.00
77    119.70
78    131.60
79    142.00
80    152.80
81    161.70
Name: I multimeter, dtype: float64

In [33]:
Slope_arr= list()
Inter_arr= list()
#reg_pred = list()
for i in range(len(csv_list)):
    for j in csv_list[i]:
        Slope_arr.append(Slope_I[i])
        Inter_arr.append(Inter_I[i])

In [35]:
Idata['I pred']=Idata['I ADC'].multiply(Slope_arr)+Inter_arr

In [36]:
E_arr=list()
for i in range(len(Idata)):
    E_arr.append(abs(Idata['I pred'][i]-Idata['I multimeter'][i])/Idata['I multimeter'][i]*100)
Idata['Error']=E_arr

In [42]:
for i in gain_list:
    print(np.average(Idata["Error"].loc[Idata['G']==i]))


12.28273775912119
5.137404840812986
0.7829953148035598
1.687920115794753


In [45]:
Inter_I

[-294.5419692528976, -35.71868100754, 8.127347522120658, -68.35364366115296]

In [37]:
import plotly as ply

In [39]:
fig = ply.graph_objects.Figure()
# Create and style traces
fig.add_trace(ply.graph_objects.Scatter(
    x=Idata["I multimeter"].loc[Idata['G'] == 1.0], y=Idata["Error"].loc[Idata['G'] == 1.0], name="1x"))
fig.add_trace(ply.graph_objects.Scatter(
    x=Idata["I multimeter"].loc[Idata['G'] == 8.0], y=Idata["Error"].loc[Idata['G'] == 8.0], name="8x"))
fig.add_trace(ply.graph_objects.Scatter(
    x=Idata["I multimeter"].loc[Idata['G'] == 4.0], y=Idata["Error"].loc[Idata['G'] == 4.0], name="4x"))
fig.add_trace(ply.graph_objects.Scatter(
    x=Idata["I multimeter"].loc[Idata['G'] == 2.0], y=Idata["Error"].loc[Idata['G'] == 2.0], name="2x"))
fig.update_layout(title='Error Pengukuran arus pada setiap gain',
                  xaxis_title='Arus (mA)',
                  yaxis_title='Error (%)')


In [4]:
(2**15*0.5627572747185379*0.0625-294.5419692528976)

857.9849293706679